In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

%run 1.tokenizer.ipynb

tokenizer

In [2]:
%run 2.dataset.ipynb


def f(data):
    data = [i['text'] for i in data]
    data = tokenizer(data, device=device)

    data['labels'] = data['input_ids'].clone()
    select = data['labels'] == tokenizer.pad_token_id
    data['labels'][select] = -100

    return data


loader = get_loader(f, negative_label=False, with_answer=True)

len(loader), next(iter(loader))

(62500,
 {'input_ids': tensor([[ 0,  8,  7,  6,  6, 14,  8, 13, 13,  8, 18, 13,  7,  5, 10,  1,  2],
          [ 0,  5, 12, 11,  7, 14, 10, 13,  9, 11, 18, 12, 12,  7,  4,  1,  2],
          [ 0, 11,  6, 10, 14, 12, 13,  4,  4, 18, 13, 10,  6, 10,  1,  2,  2],
          [ 0,  8, 13,  4, 12, 14, 13,  6,  6, 13, 18,  5,  8,  5,  7, 11,  1],
          [ 0, 10,  7,  9, 13, 14, 12,  6, 10,  8, 18,  5,  8, 10,  6,  7,  1],
          [ 0,  7, 12,  9, 14,  6,  7, 11,  4, 18,  6, 11,  9,  9,  1,  2,  2],
          [ 0, 10, 12, 11,  7, 14,  8,  8,  7,  4, 18,  5,  5,  7,  4,  7,  1],
          [ 0,  5,  6,  5,  7, 14,  6, 13,  7,  9, 18,  8,  5,  8, 12,  1,  2],
          [ 0,  8, 12, 10, 14, 13,  5, 10, 11, 18, 13, 10,  9,  7,  1,  2,  2],
          [ 0, 13,  7,  7, 13, 14,  9,  8, 13, 12, 18,  5,  8, 12,  7, 11,  1],
          [ 0,  9, 11,  6, 14,  6,  7, 12,  5, 18,  6, 13,  9,  7,  1,  2,  2],
          [ 0, 13,  8,  7, 12, 14, 10, 11, 11,  4, 18,  5, 10,  6,  4, 12,  1],
          [ 0, 11, 

In [3]:
%run 3.model.ipynb

model_actor = Gemma3Actor(len(tokenizer), tokenizer.pad_token_id).to(device)

sum(i.numel() for i in model_actor.parameters()) / 1_0000_0000

2.01442304

In [4]:
optimizer = torch.optim.Adam(model_actor.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,
                                              base_lr=1e-5,
                                              max_lr=1e-4,
                                              step_size_up=1000)

for i, data in enumerate(loader):
    loss, _ = model_actor(**data)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    if i % 1000 == 0:
        print(i, len(loader), loss.item())
        prompt = data['input_ids'][0]
        eq = prompt.tolist().index(tokenizer.eq_token_id) + 1
        chosen = prompt[eq:]
        prompt = prompt[:eq]

        gen = generate(model_actor,
                       input_ids=prompt.unsqueeze(0),
                       pad_token_id=tokenizer.pad_token_id,
                       eos_token_id=tokenizer.eos_token_id)
        gen = gen[0, eq:]

        print({
            'prompt': tokenizer.decode(prompt),
            'chosen': tokenizer.decode(chosen),
            'gen': tokenizer.decode(gen)
        })

torch.save(model_actor.to('cpu'), 'model/actor')

0 62500 3.2236034870147705
{'prompt': 'B120+7047=', 'chosen': '7167E', 'gen': 'B46B46B46B46B4'}
1000 62500 1.8045706748962402
{'prompt': 'B3684+5756=', 'chosen': '9440E', 'gen': '10000E'}
2000 62500 1.661000370979309
{'prompt': 'B6385+4142=', 'chosen': '10527E', 'gen': '10311E'}
3000 62500 1.6442676782608032
{'prompt': 'B5813+8349=', 'chosen': '14162E', 'gen': '14023E'}
4000 62500 1.5948718786239624
{'prompt': 'B4606+7036=', 'chosen': '11642E', 'gen': '11582E'}
5000 62500 1.5552464723587036
{'prompt': 'B8374+1550=', 'chosen': '9924E', 'gen': '9924E'}
6000 62500 1.3911489248275757
{'prompt': 'B5577+8832=', 'chosen': '14409E', 'gen': '14409E'}
7000 62500 1.360396385192871
{'prompt': 'B9739+2684=', 'chosen': '12423E', 'gen': '12423E'}
8000 62500 1.2600985765457153
{'prompt': 'B1153+5148=', 'chosen': '6301E', 'gen': '6301E'}
9000 62500 1.306397557258606
{'prompt': 'B6867+73=', 'chosen': '6940E', 'gen': '6330E'}
10000 62500 1.2741001844406128
{'prompt': 'B3473+510=', 'chosen': '3983E', 'gen